# Google Cloud with langchain


<h3>Tracking LangChain Executions with Aim</h3>

In this notebook we will explore three usage scenarios. To start off, we will install the necessary packages and import certain modules. Subsequently, we will configure two environment variables that can be established either within the Python script or through the terminal.

In [2]:
%pip install -U aim --upgrade
%pip install -U langchain
%pip install -U VertexAI
%pip install -U google-search-results
%pip install -U google-cloud-secret-manager==2.16.1 google-crc32c==1.5.0
%pip install -U wikipedia

  Using cached SQLAlchemy-1.4.48-cp310-cp310-macosx_12_0_arm64.whl
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.16
    Uninstalling SQLAlchemy-2.0.16:
      Successfully uninstalled SQLAlchemy-2.0.16
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.6.28 requires sqlalchemy>=2.0.15, but you have sqlalchemy 1.4.48 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=15df8e15730af145a9d31b9

In [1]:
PROJECT_ID="ml-demo-384110"

In [ ]:
PROJECT_ID=!gcloud config get-value project

print(f"PROJECT_ID: {PROJECT_ID}")

In [2]:
import os
from datetime import datetime

from langchain.llms import VertexAI
from langchain.callbacks import AimCallbackHandler, StdOutCallbackHandler

In [3]:
from google.cloud import secretmanager
import google_crc32c


def access_secret_version(
    project_id: str, secret_id: str, version_id: str
) -> str:
    """
    Access the payload for the given secret version if one exists. The version
    can be a version number as a string (e.g. "5") or an alias (e.g. "latest").
    """
    VERBOSE = False
    
    # Import the Secret Manager client library.
    from google.cloud import secretmanager

    # Create the Secret Manager client.
    client = secretmanager.SecretManagerServiceClient()

    # Build the resource name of the secret version.
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

    # Access the secret version.
    response = client.access_secret_version(request={"name": name})

    # Verify payload checksum.
    crc32c = google_crc32c.Checksum()
    crc32c.update(response.payload.data)
    if response.payload.data_crc32c != int(crc32c.hexdigest(), 16):
        print("Data corruption detected.")
        return response

    # Print the secret payload.
    #
    # WARNING: Do not print the secret in a production environment - this
    # snippet is showing how to access the secret material.
    payload = response.payload.data.decode("UTF-8")
    if VERBOSE == True:
        print(f"Plaintext: {payload}")
    
    return payload

print(PROJECT_ID)

CSE_ID = access_secret_version(PROJECT_ID, "CSE_ID", "latest")
GOOGLE_API_KEY = access_secret_version(PROJECT_ID, "GOOGLE_API_KEY", "latest")


ml-demo-384110


In [4]:
import os

os.environ["GOOGLE_API_KEY"]= GOOGLE_API_KEY
os.environ["GPLACES_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_CSE_ID"] = CSE_ID

The event methods of `AimCallbackHandler` accept the LangChain module or agent as input and log at least the prompts and generated results, as well as the serialized version of the LangChain module, to the designated Aim run.

In [5]:
session_group = datetime.now().strftime("%m.%d.%Y_%H.%M.%S")
aim_callback = AimCallbackHandler(
    repo=".",
    experiment_name="scenario 1:  LLM",
)

callbacks = [StdOutCallbackHandler(), aim_callback]
llm = VertexAI( callbacks=callbacks)

The `flush_tracker` function is used to record LangChain assets on Aim. By default, the session is reset rather than being terminated outright.

<h3>Scenario 1</h3> In the first scenario, we will use VertexAI LLM.

<h3>Scenario 2</h3> The third scenario involves an agent with tools.

In [26]:
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

In [ ]:
from langchain.tools import GooglePlacesTool

places = GooglePlacesTool()

In [ ]:
%pip install llama_index

In [ ]:
from llama_index import download_loader

GoogleCalendarReader = download_loader('GoogleCalendarReader')

loader = GoogleCalendarReader()
documents = loader.load_data()

In [8]:
from llama_index import GPTVectorStoreIndex, download_loader

GoogleCalendarReader = download_loader('GoogleCalendarReader')

loader = GoogleCalendarReader()
documents = loader.load_data()
index = GPTVectorStoreIndex.from_documents(documents)
index.query('When am I meeting Gordon?')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1008225662928-qione7umqccts4i3r7rdiph8nksjh17k.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51289%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=TUvFxegOpIhbnPBHEyvEKlm4uAigsT&access_type=offline


KeyboardInterrupt: 

In [9]:
"""Google Calendar reader."""

import datetime
import os
from typing import Any, List, Optional, Union

from llama_index.readers.base import BaseReader
from llama_index.readers.schema.base import Document

SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

# Copyright 2018 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


class GoogleCalendarReader(BaseReader):
    """Google Calendar reader.
    Reads events from Google Calendar
    """

    def load_data(
        self,
        number_of_results: Optional[int] = 100,
        start_date: Optional[Union[str, datetime.date]] = None,
    ) -> List[Document]:

        """Load data from user's calendar.
        Args:
            number_of_results (Optional[int]): the number of events to return. Defaults to 100.
            start_date (Optional[Union[str, datetime.date]]): the start date to return events from. Defaults to today.
        """

        from googleapiclient.discovery import build

        credentials = self._get_credentials()
        service = build("calendar", "v3", credentials=credentials)

        if start_date is None:
            start_date = datetime.date.today()
        elif isinstance(start_date, str):
            start_date = datetime.date.fromisoformat(start_date)

        start_datetime = datetime.datetime.combine(start_date, datetime.time.min)
        start_datetime_utc = start_datetime.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

        events_result = (
            service.events()
            .list(
                calendarId="primary",
                timeMin=start_datetime_utc,
                maxResults=number_of_results,
                singleEvents=True,
                orderBy="startTime",
            )
            .execute()
        )

        events = events_result.get("items", [])

        if not events:
            return []

        results = []
        for event in events:
            if "dateTime" in event["start"]:
                start_time = event["start"]["dateTime"]
            else:
                start_time = event["start"]["date"]

            if "dateTime" in event["end"]:
                end_time = event["end"]["dateTime"]
            else:
                end_time = event["end"]["date"]

            event_string = f"Status: {event['status']}, "
            event_string += f"Summary: {event['summary']}, "
            event_string += f"Start time: {start_time}, "
            event_string += f"End time: {end_time}, "

            organizer = event.get("organizer", {})
            display_name = organizer.get("displayName", "N/A")
            email = organizer.get("email", "N/A")
            if display_name != "N/A":
                event_string += f"Organizer: {display_name} ({email})"
            else:
                event_string += f"Organizer: {email}"

            results.append(Document(event_string))

        return results

    def _get_credentials(self) -> Any:
        """Get valid user credentials from storage.
        The file token.json stores the user's access and refresh tokens, and is
        created automatically when the authorization flow completes for the first
        time.
        Returns:
            Credentials, the obtained credential.
        """
        from google.auth.transport.requests import Request
        from google.oauth2.credentials import Credentials
        from google_auth_oauthlib.flow import InstalledAppFlow

        creds = None
        if os.path.exists("token.json"):
            creds = Credentials.from_authorized_user_file("token.json", SCOPES)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    "credentials.json", SCOPES
                )
                creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open("token.json", "w") as token:
                token.write(creds.to_json())

        return creds


if __name__ == "__main__":
    reader = GoogleCalendarReader()
    print(reader.load_data())

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1008225662928-qione7umqccts4i3r7rdiph8nksjh17k.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51320%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=8CoxfDHJOJoe0uCvMTP3MarXO6hfEa&access_type=offline


KeyboardInterrupt: 